In [1]:
%load_ext autoreload
%autoreload 2
import notebook_setup

Project root added to path: d:\google_drive\Мой диск\projects\BRepMocAutoencoder


проверим сырые данные

In [2]:
from src.config import PROCESSED_DATA_DIR
import numpy as np
from collections import defaultdict

BREP_NPZ_DIR = PROCESSED_DATA_DIR / "features" / "brep"
npz_files = list(BREP_NPZ_DIR.glob("*.npz"))

print(f"Всего файлов: {len(npz_files)}\n")
print("=" * 80)

# Собираем статистику по размерностям признаков
edge_dims = defaultdict(list)
face_dims = defaultdict(list)
coedge_dims = defaultdict(list)

for i, npz_path in enumerate(npz_files):
    with np.load(npz_path) as data:
        edge_shape = data['edge_features'].shape
        face_shape = data['face_features'].shape
        coedge_shape = data['coedge_features'].shape

        # Сохраняем размерность признаков (второе измерение)
        edge_dims[edge_shape[1]].append(npz_path.name)
        face_dims[face_shape[1]].append(npz_path.name)
        coedge_dims[coedge_shape[1]].append(npz_path.name)

        # Выводим только файлы с необычными размерностями
        if edge_shape[1] != 10 or face_shape[1] != 7 or coedge_shape[1] != 1:
            print(f"⚠️  {npz_path.name}")
            print(f"   edges: {edge_shape}, faces: {face_shape}, coedges: {coedge_shape}")

print("\n" + "=" * 80)
print("СТАТИСТИКА ПО РАЗМЕРНОСТЯМ ПРИЗНАКОВ:")
print("=" * 80)

print("\n📊 Edge Features:")
for dim, files in sorted(edge_dims.items()):
    print(f"   Размерность {dim}: {len(files)} файлов")
    if len(files) <= 5:
        for f in files:
            print(f"      - {f}")

print("\n📊 Face Features:")
for dim, files in sorted(face_dims.items()):
    print(f"   Размерность {dim}: {len(files)} файлов")
    if len(files) <= 5:
        for f in files:
            print(f"      - {f}")

print("\n📊 Coedge Features:")
for dim, files in sorted(coedge_dims.items()):
    print(f"   Размерность {dim}: {len(files)} файлов")
    if len(files) <= 5:
        for f in files:
            print(f"      - {f}")

# Проверка на несоответствия
print("\n" + "=" * 80)
if len(edge_dims) > 1 or len(face_dims) > 1 or len(coedge_dims) > 1:
    print("❌ ПРОБЛЕМА: Обнаружены файлы с РАЗНЫМИ размерностями признаков!")
    print("   Это приведёт к ошибкам при батчинге в PyTorch Geometric.")
    print("\n   Решение: Переобработайте датасет с единой feature_schema.")
else:
    print("✅ ВСЕ ФАЙЛЫ ИМЕЮТ ОДИНАКОВЫЕ РАЗМЕРНОСТИ ПРИЗНАКОВ!")
    print(f"   Edge features: {list(edge_dims.keys())[0]}")
    print(f"   Face features: {list(face_dims.keys())[0]}")
    print(f"   Coedge features: {list(coedge_dims.keys())[0]}")

2025-10-06 01:11:01.197 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: D:\google_drive\Мой диск\projects\BRepMocAutoencoder


Всего файлов: 132


СТАТИСТИКА ПО РАЗМЕРНОСТЯМ ПРИЗНАКОВ:

📊 Edge Features:
   Размерность 10: 132 файлов

📊 Face Features:
   Размерность 7: 132 файлов

📊 Coedge Features:
   Размерность 1: 132 файлов

✅ ВСЕ ФАЙЛЫ ИМЕЮТ ОДИНАКОВЫЕ РАЗМЕРНОСТИ ПРИЗНАКОВ!
   Edge features: 10
   Face features: 7
   Coedge features: 1


создадим датасет и преобразум данные в тензоры

In [3]:
import os
import json
import torch
import numpy as np
from torch.utils.data import Dataset

def standardize_features(feature_tensor, stats):

    means = np.array([s["mean"] for s in stats])
    sds = np.array([s["standard_deviation"] for s in stats])
    eps = 1e-7
    assert np.all(sds > eps), "Feature has zero standard deviation"
    means_x = np.expand_dims(means, axis=0)
    sds_x = np.expand_dims(sds, axis=0)
    feature_tensor_zero_mean = feature_tensor - means_x
    feature_tensor_standardized = feature_tensor_zero_mean / sds_x
    return feature_tensor_standardized.astype(np.float32)

def standarize_data(data, feature_standardization):
    data["face_features"] = standardize_features(data["face_features"], feature_standardization["face_features"])
    data["edge_features"] = standardize_features(data["edge_features"], feature_standardization["edge_features"])
    data["coedge_features"] = standardize_features(data["coedge_features"], feature_standardization["coedge_features"])
    return data


class BrepNetDataset(Dataset):
    def __init__(self, json_path, feats_brep_dir, split="training_set"):
        with open(json_path, encoding="utf-8") as f:
            stats = json.load(f)
        self.files = stats[split]
        self.feature_standardization = stats["feature_standardization"]
        self.split = split
        self.brep_dir = feats_brep_dir

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name = self.files[idx]
        brep_path = os.path.join(self.brep_dir, file_name + ".npz")

        D = np.load(brep_path, allow_pickle=True)

        data_np = {
            "vertex": D["vertex"].astype(np.float32),
            "edge_features": D["edge_features"].astype(np.float32),
            "face_features": D["face_features"].astype(np.float32),
            "coedge_features": D["coedge_features"].astype(np.float32),
        }

        # Применяем стандартизацию только для обучающей выборки
        if self.split == "training_set" and self.feature_standardization is not None:
             # standarize_data ожидает словарь с определенными ключами

             data_to_standardize = {
                 "face_features": data_np["face_features"],
                 "edge_features": data_np["edge_features"],
                 "coedge_features": data_np["coedge_features"]
             }
             standardized_data = standarize_data(data_to_standardize, self.feature_standardization)
             # Обновляем стандартизированные признаки в data_np
             data_np.update(standardized_data)

        return {
            "name": file_name,
            "vertices": torch.from_numpy(data_np["vertex"]),
            "edges": torch.from_numpy(data_np["edge_features"]),
            "faces": torch.from_numpy(data_np["face_features"]),
            "edge_to_vertex": torch.from_numpy(D["edge_to_vertex"].astype(np.int64)),
            "face_to_edge": torch.from_numpy(D["face_to_edge"].astype(np.int64)),
            "face_to_face": torch.from_numpy(D["face_to_face"].astype(np.int64)),
            "sdf_uv": torch.from_numpy(D["uv_faces"].astype(np.float32)),
            "sdf_vals": torch.from_numpy(D["sdf_faces"].astype(np.float32))
        }

In [4]:
import torch
from src.config import PROCESSED_DATA_DIR
STATS_BREPNET = PROCESSED_DATA_DIR / "dataset_stats.json"

BREP_NPZ_DIR = PROCESSED_DATA_DIR / "features" / "brep"

train_dataset = BrepNetDataset(STATS_BREPNET, BREP_NPZ_DIR, split="training_set")
val_dataset = BrepNetDataset(STATS_BREPNET, BREP_NPZ_DIR, split="validation_set")
test_dataset = BrepNetDataset(STATS_BREPNET, BREP_NPZ_DIR, split="test_set")

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")
sample_dataset = train_dataset[3]
for k, v in sample_dataset.items():
    print(f"{k}: {tuple(v.shape) if isinstance(v, torch.Tensor) else type(v)}")

Train samples: 94, Val samples: 24, Test samples: 14
name: <class 'str'>
vertices: (339, 3)
edges: (521, 10)
faces: (176, 7)
edge_to_vertex: (2, 521)
face_to_edge: (2, 1029)
face_to_face: (2, 495)
sdf_uv: (176, 500, 2)
sdf_vals: (176, 500)


In [5]:
import torch
import numpy as np

def augment_brep_data(data, 
                      feature_noise_std=0.05,
                      feature_dropout_prob=0.1,
                      feature_scale_range=(0.9, 1.1)):
    """
    Применяет СТОХАСТИЧЕСКИЕ аугментации к признакам BRep.
    
    ВАЖНО: Каждый вызов создаёт РАЗНЫЕ случайные изменения!
    """
    # Создаем новый словарь и клонируем тензоры
    augmented_data = {}
    for k, v in data.items():
        if isinstance(v, torch.Tensor):
            augmented_data[k] = v.clone()
        else:
            augmented_data[k] = v

    # 1. Аугментация вершин (минимальная)
    if 'vertices' in augmented_data:
        vertices = augmented_data['vertices']
        if feature_noise_std > 0:
            # Небольшой jitter для вершин
            jitter = torch.randn_like(vertices) * feature_noise_std * 0.05
            vertices = vertices + jitter
            augmented_data['vertices'] = vertices

    # 2. Аугментация признаков рёбер
    if 'edges' in augmented_data:
        edges = augmented_data['edges']
        
        # 2.1 Gaussian noise
        if feature_noise_std > 0:
            edge_noise = torch.randn_like(edges) * feature_noise_std
            edges = edges + edge_noise
        
        # 2.2 Feature dropout (зануление случайных признаков)
        if feature_dropout_prob > 0:
            edge_mask = (torch.rand_like(edges) > feature_dropout_prob).float()
            edges = edges * edge_mask
        
        # 2.3 Масштабирование
        if feature_scale_range is not None:
            scale = np.random.uniform(feature_scale_range[0], feature_scale_range[1])
            edges = edges * scale
        
        # Важно: сохраняем изменённые рёбра
        augmented_data['edges'] = edges

    # 3. Аугментация признаков граней
    if 'faces' in augmented_data:
        faces = augmented_data['faces']
        
        # 3.1 Gaussian noise
        if feature_noise_std > 0:
            face_noise = torch.randn_like(faces) * feature_noise_std
            faces = faces + face_noise
        
        # 3.2 Feature dropout
        if feature_dropout_prob > 0:
            face_mask = (torch.rand_like(faces) > feature_dropout_prob).float()
            faces = faces * face_mask
        
        # 3.3 Масштабирование
        if feature_scale_range is not None:
            scale = np.random.uniform(feature_scale_range[0], feature_scale_range[1])
            faces = faces * scale
        
        # Важно: сохраняем изменённые грани
        augmented_data['faces'] = faces

    return augmented_data

In [6]:

print("🧪 ТЕСТИРОВАНИЕ АУГМЕНТАЦИЙ")
print("=" * 80)

# Берём один образец
sample = train_dataset[0]

print(f"\n📊 Исходные данные:")
print(f"   vertices: {sample['vertices'].shape}, mean={sample['vertices'].mean():.4f}")
print(f"   edges: {sample['edges'].shape}, mean={sample['edges'].mean():.4f}")
print(f"   faces: {sample['faces'].shape}, mean={sample['faces'].mean():.4f}")

# Создаём две аугментации
aug1 = augment_brep_data(sample)
aug2 = augment_brep_data(sample)

print(f"\n📊 Аугментация 1:")
print(f"   vertices: mean={aug1['vertices'].mean():.4f}")
print(f"   edges: mean={aug1['edges'].mean():.4f}")
print(f"   faces: mean={aug1['faces'].mean():.4f}")

print(f"\n📊 Аугментация 2:")
print(f"   vertices: mean={aug2['vertices'].mean():.4f}")
print(f"   edges: mean={aug2['edges'].mean():.4f}")
print(f"   faces: mean={aug2['faces'].mean():.4f}")

# Вычисляем разницу между аугментациями
vertex_diff = (aug1['vertices'] - aug2['vertices']).abs().mean().item()
edge_diff = (aug1['edges'] - aug2['edges']).abs().mean().item()
face_diff = (aug1['faces'] - aug2['faces']).abs().mean().item()

print("\n" + "=" * 80)
print("🔍 РАЗНИЦА МЕЖДУ АУГМЕНТАЦИЯМИ:")
print("=" * 80)
print(f"   vertex_diff: {vertex_diff:.6f}")
print(f"   edge_diff:   {edge_diff:.6f}")
print(f"   face_diff:   {face_diff:.6f}")

print("\n" + "=" * 80)
print("✅ ОЦЕНКА:")
print("=" * 80)

# Оценка
if vertex_diff < 0.001:
    print("❌ vertex_diff СЛИШКОМ МАЛЕНЬКИЙ! Аугментации вершин не работают.")
else:
    print(f"✅ vertex_diff OK ({vertex_diff:.6f})")

if edge_diff < 0.01:
    print("❌ edge_diff СЛИШКОМ МАЛЕНЬКИЙ! Аугментации рёбер не работают.")
elif edge_diff > 0.5:
    print(f"⚠️  edge_diff СЛИШКОМ БОЛЬШОЙ! Аугментации слишком сильные ({edge_diff:.6f})")
else:
    print(f"✅ edge_diff OK ({edge_diff:.6f})")

if face_diff < 0.01:
    print("❌ face_diff СЛИШКОМ МАЛЕНЬКИЙ! Аугментации граней не работают.")
elif face_diff > 0.5:
    print(f"⚠️  face_diff СЛИШКОМ БОЛЬШОЙ! Аугментации слишком сильные ({face_diff:.6f})")
else:
    print(f"✅ face_diff OK ({face_diff:.6f})")

print("\n" + "=" * 80)
print("🎯 РЕКОМЕНДАЦИИ:")
print("=" * 80)
print("Оптимальные значения для контрастного обучения:")
print("   vertex_diff: 0.002 - 0.01")
print("   edge_diff:   0.05 - 0.20")
print("   face_diff:   0.05 - 0.20")

🧪 ТЕСТИРОВАНИЕ АУГМЕНТАЦИЙ

📊 Исходные данные:
   vertices: torch.Size([297, 3]), mean=0.0884
   edges: torch.Size([459, 10]), mean=-0.0217
   faces: torch.Size([157, 7]), mean=-0.0209

📊 Аугментация 1:
   vertices: mean=0.0884
   edges: mean=-0.0182
   faces: mean=-0.0174

📊 Аугментация 2:
   vertices: mean=0.0883
   edges: mean=-0.0213
   faces: mean=-0.0276

🔍 РАЗНИЦА МЕЖДУ АУГМЕНТАЦИЯМИ:
   vertex_diff: 0.002787
   edge_diff:   0.165584
   face_diff:   0.151854

✅ ОЦЕНКА:
✅ vertex_diff OK (0.002787)
✅ edge_diff OK (0.165584)
✅ face_diff OK (0.151854)

🎯 РЕКОМЕНДАЦИИ:
Оптимальные значения для контрастного обучения:
   vertex_diff: 0.002 - 0.01
   edge_diff:   0.05 - 0.20
   face_diff:   0.05 - 0.20


In [7]:
import torch
import matplotlib.pyplot as plt

print("🧪 ТЕСТ: Стохастичность аугментаций (10 прогонов)")
print("=" * 80)

sample = train_dataset[0]

# Создаём 10 аугментаций
vertex_diffs = []
edge_diffs = []
face_diffs = []

for i in range(10):
    aug1 = augment_brep_data(sample)
    aug2 = augment_brep_data(sample)
    
    vertex_diff = (aug1['vertices'] - aug2['vertices']).abs().mean().item()
    edge_diff = (aug1['edges'] - aug2['edges']).abs().mean().item()
    face_diff = (aug1['faces'] - aug2['faces']).abs().mean().item()
    
    vertex_diffs.append(vertex_diff)
    edge_diffs.append(edge_diff)
    face_diffs.append(face_diff)
    
    print(f"Прогон {i+1:2d}: vertex={vertex_diff:.6f}, edge={edge_diff:.6f}, face={face_diff:.6f}")

print("\n" + "=" * 80)
print("📊 СТАТИСТИКА (10 прогонов):")
print("=" * 80)
print(f"vertex_diff: mean={sum(vertex_diffs)/10:.6f}, std={torch.tensor(vertex_diffs).std():.6f}")
print(f"edge_diff:   mean={sum(edge_diffs)/10:.6f}, std={torch.tensor(edge_diffs).std():.6f}")
print(f"face_diff:   mean={sum(face_diffs)/10:.6f}, std={torch.tensor(face_diffs).std():.6f}")


🧪 ТЕСТ: Стохастичность аугментаций (10 прогонов)
Прогон  1: vertex=0.002777, edge=0.170074, face=0.128848
Прогон  2: vertex=0.002824, edge=0.218943, face=0.136959
Прогон  3: vertex=0.002825, edge=0.194156, face=0.133064
Прогон  4: vertex=0.002746, edge=0.164634, face=0.136113
Прогон  5: vertex=0.002777, edge=0.185291, face=0.126717
Прогон  6: vertex=0.002878, edge=0.178229, face=0.122451
Прогон  7: vertex=0.002854, edge=0.179669, face=0.150313
Прогон  8: vertex=0.002897, edge=0.168956, face=0.129059
Прогон  9: vertex=0.002830, edge=0.181359, face=0.170781
Прогон 10: vertex=0.002937, edge=0.165610, face=0.125414

📊 СТАТИСТИКА (10 прогонов):
vertex_diff: mean=0.002835, std=0.000059
edge_diff:   mean=0.180692, std=0.016360
face_diff:   mean=0.135972, std=0.014550


In [8]:
import torch
from torch_geometric.data import Batch, Data
from torch.utils.data import DataLoader


def moco_collate_fn(batch):
    """
    Функция коллации для DataLoader MoCo.
    """
    data_list_q = []
    data_list_k = []

    for idx, data_item in enumerate(batch):
        augmented_q = augment_brep_data(data_item)
        augmented_k = augment_brep_data(data_item)

        data_q = Data(
            x=augmented_q['vertices'],
            edge_attr=augmented_q['edges'],
            face_attr=augmented_q['faces'],
            edge_index=augmented_q['edge_to_vertex'],
        )

        data_k = Data(
            x=augmented_k['vertices'],
            edge_attr=augmented_k['edges'],
            face_attr=augmented_k['faces'],
            edge_index=augmented_k['edge_to_vertex'],
        )

        data_list_q.append(data_q)
        data_list_k.append(data_k)

    # Батчим только с follow_batch для face_attr
    batch_q = Batch.from_data_list(data_list_q, follow_batch=['face_attr'])
    batch_k = Batch.from_data_list(data_list_k, follow_batch=['face_attr'])

    # Вручную создаем edges_batch
    for batch_obj, data_list in [(batch_q, data_list_q), (batch_k, data_list_k)]:
        _add_edges_batch(batch_obj, data_list)
        _rename_and_add_lists(batch_obj, batch)

    return batch_q, batch_k


def simple_collate_fn(batch):
    """
    Простая функция коллации без аугментаций.
    """
    data_list = []

    for data_item in batch:
        data_obj = Data(
            x=data_item['vertices'],
            edge_attr=data_item['edges'],
            face_attr=data_item['faces'],
            edge_index=data_item['edge_to_vertex'],
        )
        data_list.append(data_obj)

    batch_data = Batch.from_data_list(data_list, follow_batch=['face_attr'])
    
    # Вручную создаем edges_batch
    _add_edges_batch(batch_data, data_list)
    _rename_and_add_lists(batch_data, batch)

    return batch_data


def _add_edges_batch(batch_obj, data_list):
    """Создает edges_batch вручную."""
    edges_batch_list = []
    for graph_idx, data in enumerate(data_list):
        num_edges = data.edge_attr.size(0)
        edges_batch_list.append(torch.full((num_edges,), graph_idx, dtype=torch.long))
    
    batch_obj.edges_batch = torch.cat(edges_batch_list)


def _rename_and_add_lists(batch_obj, original_batch):
    """Переименовывает атрибуты и добавляет списки."""
    batch_obj.vertices = batch_obj.x
    batch_obj.edges = batch_obj.edge_attr
    batch_obj.faces = batch_obj.face_attr
    batch_obj.edge_to_vertex = batch_obj.edge_index
    batch_obj.faces_batch = batch_obj.face_attr_batch
    
    # Добавляем списки из оригинального батча
    batch_obj.sdf_uv_list = [d['sdf_uv'] for d in original_batch]
    batch_obj.sdf_vals_list = [d['sdf_vals'] for d in original_batch]
    batch_obj.face_to_edge_list = [d['face_to_edge'] for d in original_batch]
    batch_obj.face_to_face_list = [d['face_to_face'] for d in original_batch]

In [9]:
print("🧪 ТЕСТ: Аугментации в батче (moco_collate_fn)")
print("=" * 80)

# Создаём маленький батч
test_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=moco_collate_fn,
    num_workers=0
)

batch_q, batch_k = next(iter(test_loader))

print(f"\n📦 Батч создан:")
print(f"   batch_q.num_graphs: {batch_q.num_graphs}")
print(f"   batch_k.num_graphs: {batch_k.num_graphs}")

# Проверяем различия между query и key
vertex_diff_batch = (batch_q.vertices - batch_k.vertices).abs().mean().item()
edge_diff_batch = (batch_q.edges - batch_k.edges).abs().mean().item()
face_diff_batch = (batch_q.faces - batch_k.faces).abs().mean().item()

print(f"\n🔍 Разница между query и key батчами:")
print(f"   vertex_diff: {vertex_diff_batch:.6f}")
print(f"   edge_diff:   {edge_diff_batch:.6f}")
print(f"   face_diff:   {face_diff_batch:.6f}")

print("\n" + "=" * 80)
if edge_diff_batch < 0.01 or face_diff_batch < 0.01:
    print("❌ ПРОБЛЕМА: Аугментации в батче не работают!")
    print("   Проверьте moco_collate_fn - возможно, аугментации не применяются.")
else:
    print("✅ Аугментации в батче работают корректно!")
    print(f"   edge_diff_batch ({edge_diff_batch:.6f}) в норме")
    print(f"   face_diff_batch ({face_diff_batch:.6f}) в норме")

🧪 ТЕСТ: Аугментации в батче (moco_collate_fn)

📦 Батч создан:
   batch_q.num_graphs: 4
   batch_k.num_graphs: 4

🔍 Разница между query и key батчами:
   vertex_diff: 0.002791
   edge_diff:   0.192943
   face_diff:   0.138578

✅ Аугментации в батче работают корректно!
   edge_diff_batch (0.192943) в норме
   face_diff_batch (0.138578) в норме


In [33]:
try:
    sample_batch_q, sample_batch_k = next(iter(test_train_loader))

    print("Информация о батче графов (запрос):")
    
    print(f"Тип объекта: {type(sample_batch_q)}")
    print(f"Количество объединенных графов в батче: {sample_batch_q.num_graphs}")
    
    print(f"Общее количество вершин в батче: {sample_batch_q.vertices.size(0)}")
    print(f"Форма объединенного тензора вершин: {sample_batch_q.vertices.shape}")
    
    print(f"Общее количество ребер (признаков ребер) в батче: {sample_batch_q.edges.size(0)}")
    print(f"Форма объединенного тензора признаков ребер: {sample_batch_q.edges.shape}")

    print(f"Общее количество граней (признаков граней) в батче: {sample_batch_q.faces.size(0)}")
    print(f"Форма объединенного тензора признаков граней: {sample_batch_q.faces.shape}")

    print(f"Общее количество связей 'edge_to_vertex' в батче: {sample_batch_q.edge_to_vertex.size(1)}")
    print(f"Форма объединенного тензора 'edge_to_vertex': {sample_batch_q.edge_to_vertex.shape}")

    print(f"Общее количество связей 'face_to_edge' в батче: {len(sample_batch_q.face_to_edge_list)}")
    print(f"Форма объединенного тензора 'face_to_edge': {sample_batch_q.face_to_edge_list[0].shape}")

    print(f"Общее количество связей 'face_to_face' в батче: {len(sample_batch_q.face_to_face_list)}")
    print(f"Форма объединенного тензора 'face_to_face': {sample_batch_q.face_to_face_list[0].shape}")

    print(f"Общее количество элементов sdf_uv в батче: {len(sample_batch_q.sdf_uv_list)}")
    print(f"Форма объединенного тензора sdf_uv: {sample_batch_q.sdf_uv_list[0].shape}")

    print(f"Общее количество элементов sdf_vals в батче: {len(sample_batch_q.sdf_vals_list)}")
    print(f"Форма объединенного тензора sdf_vals: {sample_batch_q.sdf_vals_list[0].shape}")

    print(f"Форма атрибута 'batch' (для вершин): {sample_batch_q.batch.shape}")
    print(f"Пример атрибута 'batch' (первые 10 элементов): {sample_batch_q.batch[:10]}")


except Exception as e:
     print(f"Произошла ошибка при загрузке примера батча: {e}")

Информация о батче графов (запрос):
Тип объекта: <class 'abc.DataBatch'>
Количество объединенных графов в батче: 32
Общее количество вершин в батче: 3063
Форма объединенного тензора вершин: torch.Size([3063, 3])
Общее количество ребер (признаков ребер) в батче: 4751
Форма объединенного тензора признаков ребер: torch.Size([4751, 10])
Общее количество граней (признаков граней) в батче: 1740
Форма объединенного тензора признаков граней: torch.Size([1740, 7])
Общее количество связей 'edge_to_vertex' в батче: 4751
Форма объединенного тензора 'edge_to_vertex': torch.Size([2, 4751])
Общее количество связей 'face_to_edge' в батче: 32
Форма объединенного тензора 'face_to_edge': torch.Size([2, 198])
Общее количество связей 'face_to_face' в батче: 32
Форма объединенного тензора 'face_to_face': torch.Size([2, 96])
Общее количество элементов sdf_uv в батче: 32
Форма объединенного тензора sdf_uv: torch.Size([38, 500, 2])
Общее количество элементов sdf_vals в батче: 32
Форма объединенного тензора sdf